<a href="https://colab.research.google.com/github/EricBaidoo/GhanaSegNet/blob/main/notebooks/Enhanced_GhanaSegNet_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎯 Enhanced GhanaSegNet - Food Segmentation

**Objective:** Train Enhanced GhanaSegNet to achieve 30% mIoU for food segmentation

## 📋 **Project Overview**
- **Model**: Enhanced GhanaSegNet with FPN + Advanced ASPP + Multi-Head Attention
- **Parameters**: ~10.5M
- **Backbone**: EfficientNet-B0
- **Target**: 30% mIoU (improvement over 24.37% baseline)
- **Dataset**: Ghana Food Segmentation Dataset

## 📚 **Notebook Structure**
1. **Setup & Environment** - Dependencies, paths, verification
2. **Data Loading** - Dataset preparation and loaders
3. **Model Architecture** - Enhanced GhanaSegNet implementation
4. **Training Pipeline** - Real training with optimizations
5. **Evaluation & Results** - Performance analysis
6. **Test-Time Augmentation** - Optional performance boost

---

In [ ]:
# ========================================
# 1️⃣ SETUP & ENVIRONMENT
# ========================================

# Mount Google Drive if in Colab
import sys
import os

if 'google.colab' in sys.modules:
    print("🔗 Mounting Google Drive...")
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted successfully!")
else:
    print("📍 Running locally")

In [ ]:
# Clone repository (if needed)
if 'google.colab' in sys.modules and not os.path.exists('/content/GhanaSegNet'):
    print("📥 Cloning GhanaSegNet repository...")
    !git clone https://github.com/EricBaidoo/GhanaSegNet.git /content/GhanaSegNet
    print("✅ Repository cloned successfully!")
    %cd /content/GhanaSegNet

In [ ]:
# 🔧 ENVIRONMENT VERIFICATION & SETUP

import torch
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("🔍 SYSTEM VERIFICATION")
print("="*50)

# Check CUDA availability
if torch.cuda.is_available():
    print(f"✅ CUDA available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    device = torch.device('cuda')
else:
    print("⚠️  CUDA not available - using CPU")
    device = torch.device('cpu')

# Set up paths
if 'google.colab' in sys.modules:
    PROJECT_ROOT = '/content/GhanaSegNet'
    DATA_PATH = '/content/drive/MyDrive/data'
    print("✅ Running in Google Colab")
else:
    PROJECT_ROOT = os.getcwd()
    DATA_PATH = 'data'
    print("📍 Running locally")

# Add project to Python path
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📁 Data path: {DATA_PATH}")

# Verify key files
key_files = [
    'models/ghanasegnet.py',
    'utils/losses.py', 
    'utils/metrics.py',
    'data/dataset_loader.py'
]

missing_files = []
for file_path in key_files:
    full_path = os.path.join(PROJECT_ROOT, file_path)
    if os.path.exists(full_path):
        print(f"✅ {file_path}")
    else:
        print(f"❌ {file_path} - MISSING!")
        missing_files.append(file_path)

# Check dataset
if os.path.exists(DATA_PATH):
    print(f"✅ Dataset directory found")
    if os.path.exists(os.path.join(DATA_PATH, 'train')):
        print(f"✅ Train split available")
    if os.path.exists(os.path.join(DATA_PATH, 'val')):
        print(f"✅ Validation split available")
else:
    print(f"⚠️  Dataset not found at: {DATA_PATH}")

if not missing_files:
    print(f"\n🎉 SETUP COMPLETE - Ready to proceed!")
else:
    print(f"\n⚠️  Some files missing - check repository structure")

print("="*50)

---
# ========================================
# 2️⃣ DATA LOADING & PREPARATION
# ========================================

In [ ]:
# 📊 DATASET LOADING - SYNCED WITH TRAIN_BASELINES.PY

from torch.utils.data import DataLoader
from data.dataset_loader import GhanaFoodDataset

print("📊 Loading Ghana Food Dataset (synced with train_baselines.py)...")

try:
    # EXACT SAME LOADING AS train_baselines.py
    train_dataset = GhanaFoodDataset(DATA_PATH, split='train', data_root=DATA_PATH)
    val_dataset = GhanaFoodDataset(DATA_PATH, split='val', data_root=DATA_PATH)
    
    print(f"✅ Train samples: {len(train_dataset)}")
    print(f"✅ Validation samples: {len(val_dataset)}")
    
    # Create data loaders with SAME parameters as train_baselines.py
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)
    
    print(f"✅ Data loaders created successfully (synced)")
    
except Exception as e:
    print(f"❌ Primary dataset loading failed: {e}")
    print("🔄 Trying fallback method from train_baselines.py...")
    
    try:
        # Fallback method from train_baselines.py
        train_dataset = GhanaFoodDataset('data', split='train')
        val_dataset = GhanaFoodDataset('data', split='val')
        
        train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
        val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)
        
        print(f"✅ Fallback loading successful")
        print(f"✅ Train samples: {len(train_dataset)}")
        print(f"✅ Validation samples: {len(val_dataset)}")
        
    except Exception as e2:
        print(f"❌ All dataset loading methods failed: {e2}")
        print("Please check your dataset path and structure")

In [ ]:
---
# ========================================
# 3️⃣ MODEL ARCHITECTURE
# ========================================

In [ ]:
# 🏗️ ENHANCED GHANASEGNET MODEL

from models.ghanasegnet import EnhancedGhanaSegNet
from utils.losses import CombinedLoss
from utils.metrics import calculate_miou

print("🏗️ Initializing Enhanced GhanaSegNet...")

# Initialize model
model = EnhancedGhanaSegNet(num_classes=6).to(device)
num_params = sum(p.numel() for p in model.parameters())

print(f"✅ Model initialized")
print(f"📊 Parameters: {num_params/1e6:.2f}M")
print(f"🎯 Architecture: EfficientNet-B0 + FPN + Enhanced ASPP + Multi-Head Attention")

# Initialize loss function
criterion = CombinedLoss()
print(f"✅ Combined loss function ready (Dice + Focal + Boundary)")

In [ ]:
---
# ========================================
# 4️⃣ TRAINING PIPELINE
# ========================================

In [ ]:
# ⚙️ SYNCED TRAINING CONFIGURATION
# EXACTLY matches train_baselines.py enhanced_train_model function

import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau
import time
from tqdm import tqdm

print("⚙️ Setting up training configuration (SYNCED with train_baselines.py)...")

# EXACT SAME parameters as enhanced_train_model in train_baselines.py
config = {
    'epochs': 15,                    # EXACT match with train_baselines.py
    'learning_rate': 2.5e-4,        # EXACT match with train_baselines.py
    'weight_decay': 1.2e-3,         # EXACT match with train_baselines.py
    'batch_size': 8,                # EXACT match with train_baselines.py
    'num_classes': 6,
    'device': device,
    'disable_early_stopping': True,  # EXACT match with train_baselines.py
    'use_cosine_schedule': True,     # EXACT match with train_baselines.py
    'use_progressive_training': True, # EXACT match with train_baselines.py
    'mixed_precision': True,         # EXACT match with train_baselines.py
    'benchmark_mode': True,          # EXACT match with train_baselines.py
    'custom_seed': 789,              # EXACT match with train_baselines.py
    'save_path': 'checkpoints/enhanced_ghanasegnet/best_model.pth'
}

# EXACT SAME optimizer initialization as train_baselines.py
if config['use_cosine_schedule']:
    optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    # Cosine annealing with warm restarts (from train_baselines.py)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1)
    print(f"✅ Cosine annealing scheduler with warmup")
else:
    optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)

# EXACT SAME loss function as train_baselines.py
from utils.losses import CombinedLoss
criterion = CombinedLoss(alpha=0.6, aux_weight=0.4, adaptive_weights=True).to(device)
print(f"✅ Advanced boundary-aware loss function (synced)")

print(f"✅ SYNCED CONFIGURATION:")
print(f"   📊 Epochs: {config['epochs']} (matches train_baselines.py)")
print(f"   ⚡ Learning Rate: {config['learning_rate']} (matches train_baselines.py)")
print(f"   🛡️  Weight Decay: {config['weight_decay']} (matches train_baselines.py)")
print(f"   📦 Batch Size: {config['batch_size']} (matches train_baselines.py)")
print(f"   🔥 Mixed Precision: {config['mixed_precision']}")
print(f"   📈 Cosine Schedule: {config['use_cosine_schedule']}")
print(f"   🎯 Target: 30% mIoU | Realistic: 27-28% mIoU")

# Training tracking
best_val_iou = 0.0
training_history = {
    'train_loss': [],
    'val_loss': [], 
    'val_iou': [],
    'learning_rate': [],
    'epoch_time': []
}

print(f"\n🔄 Ready for training with EXACT same parameters as your working train_baselines.py!")

In [ ]:
# 🚀 SYNCED TRAINING LOOP
# EXACTLY matches the enhanced_train_model function in train_baselines.py

print("🚀 ENHANCED GHANASEGNET - AMBITIOUS 15-EPOCH TRAINING")
print("="*60)
print(f"🎯 TARGET: 30% mIoU | REALISTIC: 27-28% mIoU")
print(f"🔧 ALL OPTIMIZATIONS ACTIVE")
print("="*60)

import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from utils.metrics import compute_iou, compute_pixel_accuracy

# Set seed for reproducibility (matching train_baselines.py)
torch.manual_seed(config['custom_seed'])
if torch.cuda.is_available():
    torch.cuda.manual_seed(config['custom_seed'])

# Initialize mixed precision training (matching train_baselines.py)
scaler = GradScaler() if config['mixed_precision'] and torch.cuda.is_available() else None

# Create checkpoint directory (matching train_baselines.py)
import os
os.makedirs('checkpoints/enhanced_ghanasegnet', exist_ok=True)

# Training loop - EXACT IMPLEMENTATION from train_baselines.py
print("🔄 Beginning training (synced with train_baselines.py)...")

for epoch in range(config['epochs']):
    start_time = time.time()
    
    # ============ TRAINING PHASE ============
    model.train()
    train_loss = 0.0
    train_samples = 0
    
    train_pbar = tqdm(train_loader, desc=f"Train Epoch {epoch+1}")
    for images, masks in train_pbar:
        images, masks = images.to(device), masks.to(device)
        
        optimizer.zero_grad()
        
        # Mixed precision forward pass (EXACT match with train_baselines.py)
        if scaler:
            with autocast():
                outputs = model(images)
                if isinstance(outputs, tuple):
                    main_output, aux_outputs = outputs
                    loss = criterion(main_output, masks, aux_outputs)
                else:
                    loss = criterion(outputs, masks)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            # Regular training (EXACT match with train_baselines.py)
            outputs = model(images)
            if isinstance(outputs, tuple):
                main_output, aux_outputs = outputs
                loss = criterion(main_output, masks, aux_outputs)
            else:
                loss = criterion(outputs, masks)
            
            loss.backward()
            optimizer.step()
        
        train_loss += loss.item()
        train_samples += images.size(0)
        
        # Update progress bar
        train_pbar.set_postfix({'Loss': f'{loss.item():.4f}'})
    
    avg_train_loss = train_loss / len(train_loader)
    
    # ============ VALIDATION PHASE ============
    model.eval()
    val_loss = 0.0
    total_iou = 0.0
    total_accuracy = 0.0
    val_samples = 0
    
    with torch.no_grad():
        val_pbar = tqdm(val_loader, desc=f"Val Epoch {epoch+1}")
        for images, masks in val_pbar:
            images, masks = images.to(device), masks.to(device)
            
            # EXACT validation implementation from train_baselines.py
            if scaler:
                with autocast():
                    outputs = model(images)
                    if isinstance(outputs, tuple):
                        main_output = outputs[0]
                    else:
                        main_output = outputs
                    loss = criterion(main_output, masks)
            else:
                outputs = model(images)
                if isinstance(outputs, tuple):
                    main_output = outputs[0]
                else:
                    main_output = outputs
                loss = criterion(main_output, masks)
            
            val_loss += loss.item()
            
            # Compute metrics (EXACT match with train_baselines.py)
            iou = compute_iou(main_output, masks)
            accuracy = compute_pixel_accuracy(main_output, masks)
            
            total_iou += iou
            total_accuracy += accuracy
            val_samples += images.size(0)
            
            val_pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'IoU': f'{iou:.4f}',
                'Acc': f'{accuracy:.4f}'
            })
    
    avg_val_loss = val_loss / len(val_loader)
    avg_val_iou = total_iou / len(val_loader)
    avg_val_accuracy = total_accuracy / len(val_loader)
    
    # Learning rate scheduling (EXACT match with train_baselines.py)
    if config['use_cosine_schedule']:
        scheduler.step()
    else:
        scheduler.step(avg_val_iou)
    
    current_lr = optimizer.param_groups[0]['lr']
    epoch_time = time.time() - start_time
    
    # Check for new best (EXACT match with train_baselines.py)
    is_best = avg_val_iou > best_val_iou
    if is_best:
        best_val_iou = avg_val_iou
        # Save best model (EXACT match with train_baselines.py)
        os.makedirs('checkpoints/enhanced_ghanasegnet', exist_ok=True)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_iou': best_val_iou,
            'config': config
        }, 'checkpoints/enhanced_ghanasegnet/best_model.pth')
    
    # Store training history
    training_history['train_loss'].append(avg_train_loss)
    training_history['val_loss'].append(avg_val_loss)
    training_history['val_iou'].append(avg_val_iou)
    training_history['learning_rate'].append(current_lr)
    training_history['epoch_time'].append(epoch_time)
    
    # Progress report (EXACT match with train_baselines.py)
    current_miou_percent = avg_val_iou * 100
    print(f"\n📊 EPOCH {epoch+1}/{config['epochs']} RESULTS:")
    print(f"   Train Loss: {avg_train_loss:.4f}")
    print(f"   Val Loss: {avg_val_loss:.4f}")
    print(f"   Val IoU: {avg_val_iou:.4f} ({current_miou_percent:.2f}%)")
    print(f"   Val Accuracy: {avg_val_accuracy:.4f}")
    print(f"   Learning Rate: {current_lr:.2e}")
    print(f"   Best IoU: {best_val_iou:.4f} ({best_val_iou*100:.2f}%)")
    print(f"   Epoch Time: {epoch_time:.1f}s")
    
    if is_best:
        print(f"   🎯 NEW BEST PERFORMANCE!")
    
    # Check milestones (matching train_baselines.py)
    milestone_alerts = [25.0, 27.0, 28.0, 29.0, 30.0]
    for milestone in milestone_alerts:
        if current_miou_percent >= milestone:
            print(f"\n? MILESTONE ACHIEVED: {milestone:.1f}% mIoU!")
            if milestone >= 30.0:
                print(f"🏆 TARGET REACHED! 30% mIoU ACHIEVED AT EPOCH {epoch+1}!")
    
    # Progress toward 30% target
    progress_to_target = (current_miou_percent - 24.4) / (30.0 - 24.4) * 100
    print(f"   📈 Progress to 30% target: {progress_to_target:.1f}%")
    
    print("-" * 60)

# Final results (EXACT match with train_baselines.py)
print(f"\n🏁 ENHANCED GHANASEGNET 15-EPOCH TRAINING COMPLETE!")
print(f"="*60)
print(f"? FINAL RESULTS:")
print(f"   Best mIoU: {best_val_iou:.4f} ({best_val_iou*100:.2f}%)")
print(f"   Target: 30.00%")
print(f"   Gap: {30.0 - best_val_iou*100:+.2f} percentage points")

if best_val_iou >= 0.30:
    print(f"? TARGET ACHIEVED! 30%+ mIoU reached!")
elif best_val_iou >= 0.28:
    print(f"🎉 EXCELLENT! Within 2% of target!")
elif best_val_iou >= 0.27:
    print(f"✅ GREAT! Solid improvement achieved!")
else:
    print(f"? Results within expected range.")

In [ ]:
---
# ========================================
# 5️⃣ EVALUATION & RESULTS
# ========================================

In [ ]:
# 📊 TRAINING RESULTS VISUALIZATION

import matplotlib.pyplot as plt

print("📊 Visualizing training results...")

# Create training plots
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Training & Validation Loss
axes[0, 0].plot(training_history['train_loss'], label='Train Loss', color='blue')
axes[0, 0].plot(training_history['val_loss'], label='Val Loss', color='red')
axes[0, 0].set_title('Training & Validation Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Validation IoU
axes[0, 1].plot(training_history['val_iou'], label='Val IoU', color='green', linewidth=2)
axes[0, 1].axhline(y=0.30, color='red', linestyle='--', label='30% Target')
axes[0, 1].axhline(y=best_val_iou, color='orange', linestyle='--', label=f'Best: {best_val_iou:.3f}')
axes[0, 1].set_title('Validation IoU Progress')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('IoU')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Learning Rate Schedule
axes[1, 0].plot(training_history['learning_rate'], label='Learning Rate', color='purple')
axes[1, 0].set_title('Learning Rate Schedule')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Learning Rate')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Performance Comparison
models = ['Baseline', 'Enhanced GhanaSegNet']
performance = [baseline_miou * 100, best_val_iou * 100]
colors = ['lightblue', 'darkblue']

axes[1, 1].bar(models, performance, color=colors)
axes[1, 1].axhline(y=30, color='red', linestyle='--', label='30% Target')
axes[1, 1].set_title('Model Performance Comparison')
axes[1, 1].set_ylabel('mIoU (%)')
axes[1, 1].legend()

# Add value labels on bars
for i, v in enumerate(performance):
    axes[1, 1].text(i, v + 0.5, f'{v:.2f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Summary statistics
print(f"\n📈 TRAINING SUMMARY:")
print(f"   Total epochs: {len(training_history['val_iou'])}")
print(f"   Best epoch: {np.argmax(training_history['val_iou']) + 1}")
print(f"   Final train loss: {training_history['train_loss'][-1]:.4f}")
print(f"   Final val loss: {training_history['val_loss'][-1]:.4f}")
print(f"   Best val IoU: {best_val_iou:.4f} ({best_val_iou*100:.2f}%)")

In [ ]:
---
# ========================================
# 6️⃣ TEST-TIME AUGMENTATION (OPTIONAL)
# ========================================

In [ ]:
# 🚀 TEST-TIME AUGMENTATION BOOST
# Run this only if you want to further improve performance

print("🚀 APPLYING TEST-TIME AUGMENTATION (TTA)")
print("="*55)
print("💡 TTA can provide +1-3% mIoU improvement")
print("🔬 Uses multi-scale and flip augmentations")
print("="*55)

import torch.nn.functional as F

# Load the best trained model
print("📥 Loading best trained model...")
try:
    checkpoint = torch.load(config['save_path'])
    model.load_state_dict(checkpoint['model_state_dict'])
    base_performance = checkpoint['best_val_iou']
    print(f"✅ Loaded model with {base_performance:.4f} ({base_performance*100:.2f}%) mIoU")
except:
    print("⚠️  Using current model state")
    base_performance = best_val_iou

model.eval()

def tta_predict(model, x):
    """Apply Test-Time Augmentation"""
    predictions = []
    
    with torch.no_grad():
        # Original prediction
        pred = model(x)
        if isinstance(pred, tuple):
            pred = pred[0]
        predictions.append(F.softmax(pred, dim=1))
        
        # Horizontal flip
        x_flip = torch.flip(x, [3])
        pred_flip = model(x_flip)
        if isinstance(pred_flip, tuple):
            pred_flip = pred_flip[0]
        pred_flip = torch.flip(F.softmax(pred_flip, dim=1), [3])
        predictions.append(pred_flip)
        
        # Multi-scale predictions  
        for scale in [0.9, 1.1]:
            h, w = x.shape[2], x.shape[3]
            new_h, new_w = int(h * scale), int(w * scale)
            
            x_scaled = F.interpolate(x, size=(new_h, new_w), mode='bilinear', align_corners=False)
            pred_scaled = model(x_scaled)
            if isinstance(pred_scaled, tuple):
                pred_scaled = pred_scaled[0]
            pred_scaled = F.interpolate(pred_scaled, size=(h, w), mode='bilinear', align_corners=False)
            predictions.append(F.softmax(pred_scaled, dim=1))
    
    return torch.stack(predictions).mean(dim=0)

# Apply TTA evaluation
print("🔄 Applying TTA to validation set...")
tta_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)

all_tta_predictions = []
all_tta_targets = []

for batch_idx, (images, masks) in enumerate(tqdm(tta_loader, desc="TTA Evaluation")):
    images = images.to(device)
    masks = masks.to(device)
    
    # Apply TTA
    tta_preds = tta_predict(model, images)
    pred_masks = torch.argmax(tta_preds, dim=1)
    
    all_tta_predictions.append(pred_masks.cpu().numpy())
    all_tta_targets.append(masks.cpu().numpy())

# Calculate TTA performance
all_tta_predictions = np.concatenate(all_tta_predictions, axis=0)
all_tta_targets = np.concatenate(all_tta_targets, axis=0)
tta_miou = calculate_miou(all_tta_predictions, all_tta_targets, num_classes=6)

# Results
improvement = (tta_miou - base_performance) * 100

print(f"\n🎯 TTA RESULTS:")
print(f"📊 Base Model: {base_performance:.4f} ({base_performance*100:.2f}% mIoU)")
print(f"🚀 With TTA: {tta_miou:.4f} ({tta_miou*100:.2f}% mIoU)")
print(f"📈 Improvement: +{improvement:.2f} percentage points")

if tta_miou >= 0.30:
    print(f"🎉 EXCELLENT! TTA achieved 30% mIoU target!")
elif tta_miou >= 0.29:
    print(f"🔥 OUTSTANDING! Very close to 30% target!")
elif improvement > 1.0:
    print(f"✅ SOLID BOOST! TTA provided meaningful improvement!")
else:
    print(f"📊 TTA applied with modest improvement")

print(f"\n🔬 TTA METHODOLOGY:")
print(f"   • Horizontal flip augmentation")
print(f"   • Multi-scale testing (0.9x, 1.0x, 1.1x)")
print(f"   • Ensemble averaging")
print(f"   • Legitimate evaluation enhancement")

# Final comparison
print(f"\n🏆 FINAL PERFORMANCE SUMMARY:")
print(f"   Baseline GhanaSegNet: 24.37% mIoU")
print(f"   Enhanced GhanaSegNet: {base_performance*100:.2f}% mIoU")
print(f"   Enhanced + TTA: {tta_miou*100:.2f}% mIoU")
print(f"   Total improvement: +{(tta_miou - 0.2437)*100:.2f} percentage points")